## Start

In [12]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables

# Links:

[[installation_point_st_partner_ch]]

[[entries_installation_points_dir_partner_ch]]

[[t_subscribed_citizen_id]]

[[companies_st_partner_ch]]

[[companies_dir_partner_ch]]

In [3]:
query_text = """--sql
    CREATE TABLE db1.t_subsctibtions_by_enteries 
    (
        `report_date` Date,
        `full_address` String,
        `city` String,
        `installation_point_id` Int64,
        `flats_count` Int16,
        `flats_count_range` Int16,
        `address_uuid` String,
        `partner_uuid` String,
        `subscribed_citizen_id_count` UInt64,
        `tariff_full` String,
        `company_name` String,
        `tin` String,
        `partner_lk` String,
        `subscribtion_rate` Float64,
        `subscribtion_rate_range` Float64
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)


""


In [7]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subsctibtions_by_enteries_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 50 MINUTE TO db1.t_subsctibtions_by_enteries AS 
WITH t_entries AS (SELECT
    report_date,
    full_address,
    city,
    installation_point_id,
    flats_count,
    flats_count_full AS flats_count_range,
    address_uuid,
    partner_uuid
FROM installation_point_st_partner_ch AS inst_st
LEFT JOIN entries_installation_points_dir_partner_ch using(installation_point_id)
),
t_subscribed AS(
SELECT
    report_date,
    address_uuid,
    subscribed_citizen_id AS subscribed_citizen_id_count
FROM t_subscribed_citizen_id
),
t_companies AS(
SELECT
    report_date,
    c_st.partner_uuid AS partner_uuid,
    tariff_full,
    company_name,
    tin,
    partner_lk
FROM companies_st_partner_ch as c_st
LEFT JOIN companies_dir_partner_ch using(partner_uuid)
)
--
SELECT
    t_entries.report_date AS report_date,
    full_address,
    city,
    installation_point_id,
    flats_count,
    flats_count_range,
    t_entries.address_uuid AS address_uuid,
    t_entries.partner_uuid AS partner_uuid,
    subscribed_citizen_id_count,
    tariff_full,
    company_name,
    tin,
    partner_lk,
    subscribed_citizen_id_count/flats_count*100 AS subscribtion_rate,
    subscribed_citizen_id_count/flats_count_range*100 AS subscribtion_rate_range
FROM t_entries
LEFT JOIN t_subscribed
    ON t_subscribed.report_date = t_entries.report_date
    AND t_subscribed.address_uuid = t_entries.address_uuid
LEFT JOIN t_companies
    ON t_companies.report_date = t_entries.report_date
    AND t_companies.partner_uuid = t_entries.partner_uuid
ORDER BY report_date DESC
	"""
ch.query_run(query_text)

""


In [13]:
query_text = """
    SELECT
        report_date,
        sum(subscribed_citizen_id_count)
    FROM db1.t_subsctibtions_by_enteries_mv
    group by report_date
    order by
    report_date desc
    LIMIT 10

    """
ch.query_run(query_text)

,report_date,sum(subscribed_citizen_id_count)
0,2025-09-10,0
1,2025-09-09,96013
2,2025-09-08,95898
3,2025-09-07,95788
4,2025-09-06,95542
5,2025-09-05,95429
6,2025-09-04,95194
7,2025-09-03,95123
8,2025-09-02,94946
9,2025-09-01,94813


In [11]:
query_text = """
    SELECT
        report_date,
        count(*)
    FROM db1.t_subsctibtions_by_enteries
    group by report_date
    order by
    report_date desc
    LIMIT 10

    """
ch.query_run(query_text)

,report_date,count()
0,2025-09-10,40188
1,2025-09-09,40169
2,2025-09-08,40139
3,2025-09-07,40103
4,2025-09-06,40103
5,2025-09-05,40102
6,2025-09-04,40067
7,2025-09-03,40048
8,2025-09-02,40019
9,2025-09-01,39989


In [4]:
query_text = """
    SELECT
        report_date,
        sum(activated_citizen_id)
    FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    group by report_date
    order by
    report_date desc
    LIMIT 10

    """
ch.query_run(query_text)

report_date,sum(activated_citizen_id)
date,i64
2025-09-07,0
2025-09-06,0
2025-09-05,0
2025-09-04,0
2025-09-03,0
2025-09-02,0
2025-09-01,0
2025-08-31,1367182
2025-08-30,1366164


In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_subsctibtions_by_enteries_mv
"""

ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,885090,"""19c49245-3dc5-4f2a-8a43-596484…"


In [ ]:
query_text = """
    DROP TABLE db1.t_subsctibtions_by_enteries_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,103057759,"""1f72e2d0-08ad-43c8-8159-887cf2…"


In [ ]:
query_text = """
    DROP TABLE db1.t_subsctibtions_by_enteries
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,3292557,"""d644f4f7-97fb-4cea-93f2-872532…"
